In [1]:
def path_for_dataset(path_train,path_seg):
    """This Function is used Extracting the file names of the images and masks in training, test and validation folders"""
    
    isic_train = next(os.walk(path_train))[2] # returns all the files "DIR."
    isic_seg_train=next(os.walk(path_seg))[2] # returns all the files "DIR."

    print("No. of images in training folder= ",len(isic_train))
    print("No. of images in test folder= ",len(isic_seg_train))
    return isic_train, isic_seg_train


In [2]:
def sorted_test(isic_train, isic_seg_train):
    """This Function is used for Sorting the data with respect to labels"""
    isic_train_sort=sorted(isic_train) # Sorting of data with respect to labels
    isic_seg_train_sort=sorted(isic_seg_train) # Sorting of data with respect to labels
    
    return isic_train_sort, isic_seg_train_sort

In [3]:
def Load_img(inp_path,isic):
    """ This function is used for Loading the images from the Training_Input_x2 folder"""
    " - Storing them with the above dimensions specified"
    " - Loading the images in Greayscale format"
    " - Normalizing the image with 255 as Normalising data by dividing it by 255 should improve activation functions performance"
    " - Sigmoid function works more efficiently with data range 0.0-1.0."
    
    X_ISIC_train= np.zeros((len(isic),img_height,img_width,1),dtype=np.float32)
    for n, id_ in tqdm_notebook(enumerate(isic), total=len(isic)): # capture all the images ids using tqdm
        img = load_img(inp_path+id_, color_mode = 'grayscale')  # Load images here
        x_img = img_to_array(img) # Convert images to array
        x_img = resize(x_img,(256,256,1),mode = 'constant',preserve_range = True)
        X_ISIC_train[n] = x_img/255 # Normalize the images
        
    return X_ISIC_train

In [4]:
def Load_segmentation(inp_path,isic):
    """ This function is used for Loading the images from the Training_GroundTruth_x2 folder"""
    " - Storing them with the above dimensions specified"
    " - Loading the images in Greayscale format"
    
    Y_ISIC_train= np.zeros((len(isic),img_height,img_width,1),dtype=np.uint8)
    for n, id_ in tqdm_notebook(enumerate(isic), total=len(isic)):
        # Load images
        img = load_img(inp_path+id_,color_mode = 'grayscale')
        x_img = img_to_array(img)
        x_img = resize(x_img,(256,256,1),mode = 'constant', preserve_range = True)
        Y_ISIC_train[n] = x_img
        
    return Y_ISIC_train

In [5]:
def load_dataset(X_ISIC_train, Y_ISIC_train):
    """This Function is used for spliting the dataset into Train, Test, and Val"""
    X_train, X_test, y_train, y_test = train_test_split(X_ISIC_train, Y_ISIC_train, test_size=0.20, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    X_train.shape
    
    return X_train, X_test, y_train, y_test, X_val, y_val

In [6]:
def pre_processing(y_train,y_test,y_val):
    """This Function is used for Pre_processing the Y labels into two categoraical format"""
    " - By calculating the quotient "
    " - By using One-Hot Encoding to Labels"
    Y_ISIC_train_sc = y_train//255
    Y_ISIC_test_sc = y_test//255
    Y_ISIC_val_sc = y_val//255
    Y_ISIC_train_cat = to_categorical(Y_ISIC_train_sc) # one hot encoding
    Y_ISIC_test_cat = to_categorical(Y_ISIC_test_sc) # one hot encoding
    Y_ISIC_val_cat = to_categorical(Y_ISIC_val_sc) # one hot encoding
    
    return Y_ISIC_train_cat, Y_ISIC_test_cat, Y_ISIC_val_cat

In [7]:
# Dice Coeffient
from keras import backend as K
def dice_coeff(y_true, y_pred, smooth=1):
    """This Function is used to gauge similarity of two samples"""
    "When applied to Boolean data, using the definition of true positive (TP), false positive (FP), and false negative (FN)"
    
    intersect = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersect
    coeff_dice = K.mean((intersect + smooth) / (union + smooth), axis=0)
    return coeff_dice

Using TensorFlow backend.


In [8]:
def dice_loss(y_true, y_pred, smooth = 1):
    """This Function returns the dice loss by subtracting the dice_coeff similarity by 1 of the two samples"""
    
    return 1 - dice_coeff(y_true, y_pred, smooth = 1)

In [9]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    """This Function is used to define the Conv2D layers by defining the filters, kernalsize and kernal initilizer"""
    " - It defines the BatchNorm of the previous layer"
    " - It defines the Activation function(relu) to be used in the Conv2D layer"
    
    # first layer
    layer = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    # It draws samples from a truncated normal distribution centered on 0 
    if batchnorm:
        layer = BatchNormalization()(layer)
    layer = Activation("relu")(layer)
    # second layer
    layer = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        layer = BatchNormalization()(layer)
    layer = Activation("relu")(layer)
    return layer


In [10]:
def generat_unet(input_img, n_filters=16, dropout=0.1, batchnorm=True):
    """This Function is used to define the Architecture of the U-Net model for the Contracting and Expanding path"""
    " - It defines the Kernal size to be 3*3"
    " - It defines the MaxPooling of (2,2) strides"
    " - It defines the DropOut layer of 0.05"
    # contracting path - reduce enoder part
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path - Decoder part
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(2, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [11]:
def loss_plot(results):
    """This Function is used for used for plotting the graph of the training and Validation loss with respect to epoch"""
    plt.figure(figsize=(8, 8))
    plt.title("Dice_Loss")
    plt.plot(results.history["loss"], label="training_loss")
    plt.plot(results.history["val_loss"], label="validation_loss")
    plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
    plt.xlabel("Epochs")
    plt.ylabel("loss")
    plt.legend();

In [12]:
def acc_plot(results):
    """This Function is used for Plotting the training and validation accuracy with respect to epochs"""
    plt.figure(figsize=(8,8))
    plt.title("Classification Accuracy")
    plt.plot(results.history["accuracy"],label="training_accuracy")
    plt.plot(results.history["val_accuracy"],label="validation_accuracy")
    plt.plot(np.argmin(results.history["val_accuracy"]),np.max(results.history["val_accuracy"]),marker="x",color="r",label="best model")
    plt.xlabel("Epochs")
    plt.legend();

In [13]:
def best_model(model,X_test,y_test):
    """This Function is used for Capturing the Best model for the epoch"""
    model.load_weights('model-ISIC.h5')
    test_preds=model.predict(X_test,verbose=1) # predict the model
    test_preds_max=np.argmax(test_preds,axis=-1) # Returns the indices of the maximum values along an axis
    n,h,w,g=y_test.shape
    test_preds_reshape=test_preds_max.reshape(n,h,w,g)
    return test_preds_reshape

In [14]:
def plot_ISIc(X, y, Y_pred,ix=None):
    """This function is used for ploting the True image vs the Predictive image from the above model"""
    if ix is None:
        ix = random.randint(0, len(X))
    else:
        ix = ix
    

    fig, ax = plt.subplots(1, 3, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='gray')
    ax[0].contour(X[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Input Image')
    
    
    ax[1].imshow(y[ix, ..., 0], cmap='gray')
    ax[1].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[1].set_title('True Image')
    
    ax[2].imshow(Y_pred[ix, ..., 0], cmap='gray')
    ax[2].contour(Y_pred[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('Predicted Image')
    